# Install libraries

In [1]:
! pip install xgboost

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


# Load libraries

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
import xgboost as xgb
import warnings

warnings.filterwarnings("ignore")

# Load data

In [3]:
# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

In [4]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



---



# Voting classifier
Train a few classifiers and aggregate their predictions. Then predict the class that gets the most votes.


In [5]:
# Instantiate the individual models
log_clf = LogisticRegression()
svm_clf = SVC(probability=True)
tree_clf = DecisionTreeClassifier()

In [30]:
# Create the voting classifier
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('svc', svm_clf), ('tree', tree_clf)], voting='soft' )

In [7]:
# Fit the model
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('svc', SVC(probability=True)),
                             ('tree', DecisionTreeClassifier())],
                 voting='soft')

In [8]:
# Evaluate the model
print("Voting Classifier Accuracy:", voting_clf.score(X_test, y_test))

Voting Classifier Accuracy: 1.0




---



# Stacking

Instead of using trivial functions (such as hard voting) to aggregate the predictions of all predictors in an ensemble, why don't we train a model to perform this aggregation?


In [9]:
# Define base models
base_models = [
                ('lr', LogisticRegression()),
                ('svc', SVC(probability=True)),
                ('tree', DecisionTreeClassifier())
            ]

In [10]:
# Stacking classifier
stack_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression()
)

In [11]:
# Fit the model
stack_clf.fit(X_train, y_train)

StackingClassifier(estimators=[('lr', LogisticRegression()),
                               ('svc', SVC(probability=True)),
                               ('tree', DecisionTreeClassifier())],
                   final_estimator=LogisticRegression())

In [12]:
# Evaluate the model
print("Stacking Classifier Accuracy:", stack_clf.score(X_test, y_test))

Stacking Classifier Accuracy: 1.0




---



# Bagging with Decision Trees

Use the same classifier for each predictor, but train them on different random subsets of the training set

Bagging: Sampling is performed with replacement


In [31]:
# Bagging classifier
bag_clf = BaggingClassifier( DecisionTreeClassifier(), n_estimators=500, max_samples=100, bootstrap=True, n_jobs=-1 )

In [32]:
# Fit the model
bag_clf.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1)

In [15]:
# Evaluate the model
print("Bagging Classifier Accuracy:", bag_clf.score(X_test, y_test))

Bagging Classifier Accuracy: 1.0


# Pasting with Decision Trees
Pasting is similar to bagging; the only difference is setting `bootstrap=False` to use sampling without replacement.



In [33]:
# Pasting classifier (similar to the Bagging code, just switch `bootstrap` to False)
bag_clf = BaggingClassifier( DecisionTreeClassifier(), n_estimators=500, max_samples=100, bootstrap=False, n_jobs=-1 )

In [34]:
# The rest of the code is the same as for bagging
bag_clf.fit(X_train, y_train)
print("Pasting Classifier Accuracy:", bag_clf.score(X_test, y_test))

Pasting Classifier Accuracy: 1.0


# Random Forest

The fundamental difference between Random Forest and bagging of Decision Trees is that in Random forests, only a subset of features are selected at random out of the total and the best split feature from the subset is used to split each node in a tree, unlike in bagging where all features are considered for splitting a node

- To simulate RF using bagging, initialize the DT as:

`dt = DecisionTreeClassifier(splitter='random')`



In [35]:
# Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100)

In [36]:
# Fit the model
rf_clf.fit(X_train, y_train)

RandomForestClassifier()

In [37]:
# Evaluate the model
print("Random Forest Classifier Accuracy:", rf_clf.score(X_test, y_test))

Random Forest Classifier Accuracy: 1.0




---



# Boosting with AdaBoost

- Particularly effective for binary classification tasks
- Focuses on converting weak learners into strong ones in a sequential manner.

In [38]:
# AdaBoost classifier
ada_clf = AdaBoostClassifier( DecisionTreeClassifier(max_depth=1), n_estimators=200, algorithm="SAMME.R", learning_rate=0.5 )

In [39]:
# Fit the model
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

In [40]:
# Evaluate the model
print("AdaBoost Classifier Accuracy:", ada_clf.score(X_test, y_test))

AdaBoost Classifier Accuracy: 0.9666666666666667


# Gradient Boosting

- Gradient Boosting is a more general and flexible boosting approach than AdaBoost
- It also builds models sequentially, but it uses the gradient of the loss function to guide the learning process


In [41]:
# Gradient Boosting classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)

In [42]:
# Fit the model
gb_clf.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=42)

In [43]:
# Evaluate the model
print("Gradient Boosting Classifier Accuracy:", gb_clf.score(X_test, y_test))

Gradient Boosting Classifier Accuracy: 0.9666666666666667


# XGBoost

XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible, and portable. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a massive improvement in speed and performance compared to traditional Gradient Boosting.

In [44]:
# XGBoost classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [45]:
# Predictions
predictions = xgb_clf.predict(X_test)

In [46]:
# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print("XGBoost Classifier Accuracy:", accuracy)

XGBoost Classifier Accuracy: 1.0
